# Predicting Molecular Properties

# Table of Contents:

**1. [Problem Definition](#id1)** <br>
**2. [Get the Data (Collect / Obtain)](#id2)** <br>
**3. [Load the Dataset](#id3)** <br>
**4. [Data Visualizations](#id4)** <br>
**5. [Data Pre-processing](#id5)** <br>
**5. [Model](#id6)** <br>
**6. [Visualization and Analysis of Results](#id7)** <br>
**7. [Submittion](#id8)** <br>
**8. [References](#ref)** <br>

<a id="id1"></a> <br> 
# **1. Problem Definition:** 

This challenge aims to predict interactions between atoms. The main task is develop an algorithm that can predict the magnetic interaction between two atoms in a molecule (i.e., the scalar coupling constant)<br>

In this competition, you will be predicting the scalar_coupling_constant between atom pairs in molecules, given the two atom types (e.g., C and H), the coupling type (e.g., 2JHC), and any features you are able to create from the molecule structure (xyz) files.

**Data**
* **train.csv** - the training set, where the first column (molecule_name) is the name of the molecule where the coupling constant originates, the second (atom_index_0) and third column (atom_index_1) is the atom indices of the atom-pair creating the coupling and the fourth column (**scalar_coupling_constant**) is the scalar coupling constant that we want to be able to predict
* **test.csv** - the test set; same info as train, without the target variable
* **sample_submission.csv** - a sample submission file in the correct format
* **structures.csv** - this file contains the same information as the individual xyz structure files, but in a single file

**Additional Data**<br>
*NOTE: additional data is provided for the molecules in Train only!*
* **scalar_coupling_contributions.csv** - The scalar coupling constants in train.csv are a sum of four terms. The first column (**molecule_name**) are the name of the molecule, the second (**atom_index_0**) and third column (**atom_index_1**) are the atom indices of the atom-pair, the fourth column indicates the **type** of coupling, the fifth column (**fc**) is the Fermi Contact contribution, the sixth column (**sd**) is the Spin-dipolar contribution, the seventh column (**pso**) is the Paramagnetic spin-orbit contribution and the eighth column (**dso**) is the Diamagnetic spin-orbit contribution.



<a id="id2"></a> <br> 
# **2. Get the Data (Collect / Obtain):** 

## All imports used in this kernel

In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from IPython.display import HTML
import json
# import altair as alt

import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import os
import time
import datetime
import json
import gc
from numba import jit

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics

from itertools import product

from IPython.display import HTML
from tqdm import tqdm_notebook as tqdm
# alt.renderers.enable('notebook')

In [2]:
work = "predicting" # testing or predicting

## All function used in this kernel

In [3]:

# def add_autoincrement(render_func):
#     # Keep track of unique <div/> IDs
#     cache = {}
#     def wrapped(chart, id="vega-chart", autoincrement=True):
#         if autoincrement:
#             if id in cache:
#                 counter = 1 + cache[id]
#                 cache[id] = counter
#             else:
#                 cache[id] = 0
#             actual_id = id if cache[id] == 0 else id + '-' + str(cache[id])
#         else:
#             if id not in cache:
#                 cache[id] = 0
#             actual_id = id
#         return render_func(chart, id=actual_id)
#     # Cache will stay outside and 
#     return wrapped
           

# @add_autoincrement
# def render(chart, id="vega-chart"):
#     """
#     Helper function to plot altair visualizations.
#     """
#     chart_str = """
#     <div id="{id}"></div><script>
#     require(["vega-embed"], function(vg_embed) {{
#         const spec = {chart};     
#         vg_embed("#{id}", spec, {{defaultStyle: true}}).catch(console.warn);
#         console.log("anything?");
#     }});
#     console.log("really...anything?");
#     </script>
#     """
#     return HTML(
#         chart_str.format(
#             id=id,
#             chart=json.dumps(chart) if isinstance(chart, dict) else chart.to_json(indent=None)
#         )
#     )
    

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    

@jit
def fast_auc(y_true, y_prob):
    """
    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc


def eval_auc(y_true, y_pred):
    """
    Fast auc eval function for lgb.
    """
    return 'auc', fast_auc(y_true, y_pred), True


def group_mean_log_mae(y_true, y_pred, types, floor=1e-9):
    """
    Fast metric computation for this competition: https://www.kaggle.com/c/champs-scalar-coupling
    Code is from this kernel: https://www.kaggle.com/uberkinder/efficient-metric
    """
    maes = (y_true-y_pred).abs().groupby(types).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()
    

def train_model_regression(X, X_test, y, params, folds, model_type='lgb', eval_metric='mae', columns=None, plot_feature_importance=False, model=None,
                               verbose=10000, early_stopping_rounds=200, n_estimators=50000):
    """
    A function to train a variety of regression models.
    Returns dictionary with oof predictions, test predictions, scores and, if necessary, feature importances.
    
    :params: X - training data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: X_test - test data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: y - target
    :params: folds - folds to split data
    :params: model_type - type of model to use
    :params: eval_metric - metric to use
    :params: columns - columns to use. If None - use all columns
    :params: plot_feature_importance - whether to plot feature importance of LGB
    :params: model - sklearn model, works only for "sklearn" model type
    
    """
    columns = X.columns if columns is None else columns
    X_test = X_test[columns]
    
    # to set up scoring parameters
    metrics_dict = {'mae': {'lgb_metric_name': 'mae',
                        'catboost_metric_name': 'MAE',
                        'sklearn_scoring_function': metrics.mean_absolute_error},
                    'group_mae': {'lgb_metric_name': 'mae',
                        'catboost_metric_name': 'MAE',
                        'scoring_function': group_mean_log_mae},
                    'mse': {'lgb_metric_name': 'mse',
                        'catboost_metric_name': 'MSE',
                        'sklearn_scoring_function': metrics.mean_squared_error}
                    }

    
    result_dict = {}
    
    # out-of-fold predictions on train data
    oof = np.zeros(len(X))
    
    # averaged predictions on train data
    prediction = np.zeros(len(X_test))
    
    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()
    
    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = n_estimators, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                    verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            
            y_pred = model.predict(X_test).reshape(-1,)
        
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric=metrics_dict[eval_metric]['catboost_metric_name'], **params,
                                      loss_function=metrics_dict[eval_metric]['catboost_metric_name'])
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        if eval_metric != 'group_mae':
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
        else:
            scores.append(metrics_dict[eval_metric]['scoring_function'](y_valid, y_pred_valid, X_valid['type']))

        prediction += y_pred    
        
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= folds.n_splits
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    
    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= folds.n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
            
            result_dict['feature_importance'] = feature_importance
        
    return result_dict
    


def train_model_classification(X, X_test, y, params, folds, model_type='lgb', eval_metric='auc', columns=None, plot_feature_importance=False, model=None,
                               verbose=10000, early_stopping_rounds=200, n_estimators=50000):
    """
    A function to train a variety of regression models.
    Returns dictionary with oof predictions, test predictions, scores and, if necessary, feature importances.
    
    :params: X - training data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: X_test - test data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: y - target
    :params: folds - folds to split data
    :params: model_type - type of model to use
    :params: eval_metric - metric to use
    :params: columns - columns to use. If None - use all columns
    :params: plot_feature_importance - whether to plot feature importance of LGB
    :params: model - sklearn model, works only for "sklearn" model type
    
    """
    columns = X.columns if columns == None else columns
    X_test = X_test[columns]
    
    # to set up scoring parameters
    metrics_dict = {'auc': {'lgb_metric_name': eval_auc,
                        'catboost_metric_name': 'AUC',
                        'sklearn_scoring_function': metrics.roc_auc_score},
                    }
    
    result_dict = {}
    
    # out-of-fold predictions on train data
    oof = np.zeros((len(X), len(set(y.values))))
    
    # averaged predictions on train data
    prediction = np.zeros((len(X_test), oof.shape[1]))
    
    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()
    
    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
        if model_type == 'lgb':
            model = lgb.LGBMClassifier(**params, n_estimators=n_estimators, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                    verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            
            y_pred_valid = model.predict_proba(X_valid)
            y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=n_estimators, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            
            y_pred = model.predict_proba(X_test)
        
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=n_estimators, eval_metric=metrics_dict[eval_metric]['catboost_metric_name'], **params,
                                      loss_function=metrics_dict[eval_metric]['catboost_metric_name'])
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        oof[valid_index] = y_pred_valid
        scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid[:, 1]))

        prediction += y_pred    
        
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= folds.n_splits
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    
    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= folds.n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
            
            result_dict['feature_importance'] = feature_importance
        
    return result_dict

# # setting up altair
# workaround = prepare_altair()
# HTML("".join((
#     "<script>",
#     workaround,
#     "</script>",
# )))

<a id="id3"></a> <br> 
# **3. Load the Dataset** 

Let's load all necessary datasets

In [4]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sub = pd.read_csv('./data/sample_submission.csv')
structures = pd.read_csv('./data/structures.csv')
scalar_coupling_contributions = pd.read_csv('./data/scalar_coupling_contributions.csv')
# magnetic_shielding = pd.read_csv('./data/magnetic_shielding_tensors.csv')
# mulliken_charge = pd.read_csv('./data/mulliken_charges.csv')
print('Train dataset shape is -> rows: {} cols:{}'.format(train.shape[0],train.shape[1]))
print('Test dataset shape is  -> rows: {} cols:{}'.format(test.shape[0],test.shape[1]))
print('Sub dataset shape is  -> rows: {} cols:{}'.format(sub.shape[0],sub.shape[1]))
print('Structures dataset shape is  -> rows: {} cols:{}'.format(structures.shape[0],structures.shape[1]))
print('Structures dataset shape is  -> rows: {} cols:{}'.format(structures.shape[0],structures.shape[1]))
print('Scalar_coupling_contributions dataset shape is  -> rows: {} cols:{}'.format(scalar_coupling_contributions.shape[0],
                                                                                   scalar_coupling_contributions.shape[1]))
# print('magnetic shielding dataset shape is  -> rows: {} cols:{}'.format(magnetic_shielding.shape[0],magnetic_shielding.shape[1]))
# print('mulliken charge dataset shape is  -> rows: {} cols:{}'.format(mulliken_charge.shape[0], mulliken_charge.shape[1]))

Train dataset shape is -> rows: 4658147 cols:6
Test dataset shape is  -> rows: 2505542 cols:5
Sub dataset shape is  -> rows: 2505542 cols:2
Structures dataset shape is  -> rows: 2358657 cols:6
Structures dataset shape is  -> rows: 2358657 cols:6
Scalar_coupling_contributions dataset shape is  -> rows: 4658147 cols:8


In [5]:
# magnetic_shielding = pd.read_csv('./data/magnetic_shielding_tensors.csv')


For an fast model/feature evaluation, get only 10% of dataset. Final submission must remove/coments this code

<a id="id4"></a> <br> 
# **4. Data Visualizations** 

In [6]:
# merging with mulliken charge and sclar coupling contributions
'''
train = pd.merge(train, mulliken_charge, left_on = ['molecule_name', 'atom_index_0'], right_on = ['molecule_name', 'atom_index'])
train = pd.merge(train, scalar_coupling_contributions, left_on = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'], 
                right_on = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'])
                '''

"\ntrain = pd.merge(train, mulliken_charge, left_on = ['molecule_name', 'atom_index_0'], right_on = ['molecule_name', 'atom_index'])\ntrain = pd.merge(train, scalar_coupling_contributions, left_on = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'], \n                right_on = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'])\n                "

In [7]:
'''train.drop('atom_index', axis = 1, inplace = True)'''

"train.drop('atom_index', axis = 1, inplace = True)"

In [8]:
'''sns.lmplot(x = 'mulliken_charge', y = 'scalar_coupling_constant', data = train, fit_reg = False, row = 'type')
plt.show()'''

"sns.lmplot(x = 'mulliken_charge', y = 'scalar_coupling_constant', data = train, fit_reg = False, row = 'type')\nplt.show()"

In [9]:
'''sns.lmplot(x = 'fc', y = 'scalar_coupling_constant', data = train, fit_reg = False)
plt.show()'''

"sns.lmplot(x = 'fc', y = 'scalar_coupling_constant', data = train, fit_reg = False)\nplt.show()"

In [10]:
'''sns.lmplot(x = 'sd', y = 'scalar_coupling_constant', data = train, fit_reg = False)
sns.lmplot(x = 'pso', y = 'scalar_coupling_constant', data = train, fit_reg = False)
sns.lmplot(x = 'dso', y = 'scalar_coupling_constant', data = train, fit_reg = False)
plt.show()'''

"sns.lmplot(x = 'sd', y = 'scalar_coupling_constant', data = train, fit_reg = False)\nsns.lmplot(x = 'pso', y = 'scalar_coupling_constant', data = train, fit_reg = False)\nsns.lmplot(x = 'dso', y = 'scalar_coupling_constant', data = train, fit_reg = False)\nplt.show()"

In [11]:
'''for types in train.type.unique():
    sns.lmplot(x = 'sd', y = 'scalar_coupling_constant', data = train.loc[train.type == types], fit_reg = False)
plt.show()'''

"for types in train.type.unique():\n    sns.lmplot(x = 'sd', y = 'scalar_coupling_constant', data = train.loc[train.type == types], fit_reg = False)\nplt.show()"

In [12]:
'''from scipy.stats import pearsonr'''

'from scipy.stats import pearsonr'

In [13]:
'''print('Calculating correlations')
df_corr = {}
corr = list()
pval = list()
colval = list()
for col in tqdm(train.columns.values[~np.isin(train.columns.values, ['id', 'molecule_name', 'type'])]):
    pears = np.abs(pearsonr(train.scalar_coupling_constant, train[col]))
    corr.append(pears[0])
    pval.append(pears[1])
    colval.append(col)
    dic  = {'corr':corr, 'pval':pval, 'column':colval}
df_corr = pd.DataFrame(dic).sort_values(by='corr', ascending=False)
print(df_corr)'''

"print('Calculating correlations')\ndf_corr = {}\ncorr = list()\npval = list()\ncolval = list()\nfor col in tqdm(train.columns.values[~np.isin(train.columns.values, ['id', 'molecule_name', 'type'])]):\n    pears = np.abs(pearsonr(train.scalar_coupling_constant, train[col]))\n    corr.append(pears[0])\n    pval.append(pears[1])\n    colval.append(col)\n    dic  = {'corr':corr, 'pval':pval, 'column':colval}\ndf_corr = pd.DataFrame(dic).sort_values(by='corr', ascending=False)\nprint(df_corr)"

It's clear from this that fc (Fermi-contact is the strongest predictor, but note that also dso, sd have strong importance. Below we'll include code to predict those with given features and then add this to the prediction

In [14]:
if work == "testing":
    n_estimators_default = 1000
elif work == "predicting":
    n_estimators_default = 6000

In [15]:
# ### COMMENT THIS TO RUN ACTUAL SUBMISSION ###
if work == "testing":
    size = round(0.10*train.shape[0])
    train = train[:size]
    test = test[:size]
    sub = sub[:size]
    structures = structures[:size]
    scalar_coupling_contributions = scalar_coupling_contributions[:size]

    print('Train dataset shape is now rows: {} cols:{}'.format(train.shape[0],train.shape[1]))
    print('Test dataset shape is now rows: {} cols:{}'.format(test.shape[0],test.shape[1]))
    print('Sub dataset shape is now rows: {} cols:{}'.format(sub.shape[0],sub.shape[1]))
    print('Structures dataset shape is now rows: {} cols:{}'.format(structures.shape[0],structures.shape[1]))
    print('Scalar_coupling_contributions dataset shape is now rows: {} cols:{}'.format(scalar_coupling_contributions.shape[0],
                                                                                       scalar_coupling_contributions.shape[1]))
    

In [16]:
train.shape

(4658147, 6)

The importante things to know is that the scalar coupling constants in train.csv are a sum of four terms. 
```
* fc is the Fermi Contact contribution
* sd is the Spin-dipolar contribution
* pso is the Paramagnetic spin-orbit contribution
* dso is the Diamagnetic spin-orbit contribution. 
```
Let's merge this into train

In [17]:
train = pd.merge(train, scalar_coupling_contributions, how = 'left',
                  left_on  = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'],
                  right_on = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'])

In [18]:
train.head(10)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,fc,sd,pso,dso
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807599999999994,83.022400000000005,0.254579,1.25862,0.272010
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000000000000,-11.034700000000001,0.352978,2.85839,-3.433600
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254799999999999,-11.032500000000001,0.352944,2.85852,-3.433870
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300000000001,-11.031900000000000,0.352934,2.85855,-3.433930
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807400000000001,83.022199999999998,0.254585,1.25861,0.272013
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.254099999999999,-11.031700000000001,0.352932,2.85856,-3.433950
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.254799999999999,-11.032400000000001,0.352943,2.85853,-3.433870
7,7,dsgdb9nsd_000001,3,0,1JHC,84.809299999999993,83.024100000000004,0.254634,1.25856,0.272012
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.254300000000001,-11.031900000000000,0.352943,2.85856,-3.433930
9,9,dsgdb9nsd_000001,4,0,1JHC,84.809500000000000,83.024299999999997,0.254628,1.25856,0.272012


In [19]:
test.head(10)

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC
5,4658152,dsgdb9nsd_000015,3,0,1JHC
6,4658153,dsgdb9nsd_000015,3,2,3JHC
7,4658154,dsgdb9nsd_000015,3,4,2JHH
8,4658155,dsgdb9nsd_000015,3,5,2JHH
9,4658156,dsgdb9nsd_000015,4,0,1JHC


`train['scalar_coupling_constant'] and scalar_coupling_contributions['fc']` quite similar

In [20]:
pd.concat(objs=[train['scalar_coupling_constant'],scalar_coupling_contributions['fc'] ],axis=1)[:10]

,scalar_coupling_constant,fc
0,84.807599999999994,83.022400000000005
1,-11.257000000000000,-11.034700000000001
2,-11.254799999999999,-11.032500000000001
3,-11.254300000000001,-11.031900000000000
4,84.807400000000001,83.022199999999998
5,-11.254099999999999,-11.031700000000001
6,-11.254799999999999,-11.032400000000001
7,84.809299999999993,83.024100000000004
8,-11.254300000000001,-11.031900000000000
9,84.809500000000000,83.024299999999997


Based in others ideais we can:<br>

- train a model to predict `fc` feature;
- add this feature to train and test and train the same model to compare performance;
- train a better model;

<a id="id5"></a> <br> 
# **5. Data Pre-processing** 

## Feature generation

I use this great kernel to get x,y,z position. https://www.kaggle.com/seriousran/just-speed-up-calculate-distance-from-benchmark

In [21]:
from tqdm import tqdm_notebook as tqdm
atomic_radius = {'H':0.38, 'C':0.77, 'N':0.75, 'O':0.73, 'F':0.71} # Without fudge factor

fudge_factor = 0.05
atomic_radius = {k:v + fudge_factor for k,v in atomic_radius.items()}
print(atomic_radius)

electronegativity = {'H':2.2, 'C':2.55, 'N':3.04, 'O':3.44, 'F':3.98}

#structures = pd.read_csv(structures, dtype={'atom_index':np.int8})

atoms = structures['atom'].values
atoms_en = [electronegativity[x] for x in tqdm(atoms)]
atoms_rad = [atomic_radius[x] for x in tqdm(atoms)]

structures['EN'] = atoms_en
structures['rad'] = atoms_rad

display(structures.head())

{'H': 0.43, 'C': 0.8200000000000001, 'N': 0.8, 'O': 0.78, 'F': 0.76}


,molecule_name,atom_index,atom,x,y,z,EN,rad
0,dsgdb9nsd_000001,0,C,-0.0126981359,1.0858041580,0.0080009958,2.55,0.82
1,dsgdb9nsd_000001,1,H,0.0021504160,-0.0060313176,0.0019761204,2.20,0.43
2,dsgdb9nsd_000001,2,H,1.0117308430,1.4637511620,0.0002765748,2.20,0.43
3,dsgdb9nsd_000001,3,H,-0.5408150690,1.4475266140,-0.8766437152,2.20,0.43
4,dsgdb9nsd_000001,4,H,-0.5238136345,1.4379326440,0.9063972942,2.20,0.43


### Chemical Bond Calculation

In [22]:
i_atom = structures['atom_index'].values
p = structures[['x', 'y', 'z']].values
p_compare = p
m = structures['molecule_name'].values
m_compare = m
r = structures['rad'].values
r_compare = r

source_row = np.arange(len(structures))
max_atoms = 28

bonds = np.zeros((len(structures)+1, max_atoms+1), dtype=np.int8)
bond_dists = np.zeros((len(structures)+1, max_atoms+1), dtype=np.float32)

print('Calculating bonds')

for i in tqdm(range(max_atoms-1)):
    p_compare = np.roll(p_compare, -1, axis=0)
    m_compare = np.roll(m_compare, -1, axis=0)
    r_compare = np.roll(r_compare, -1, axis=0)
    
    mask = np.where(m == m_compare, 1, 0) #Are we still comparing atoms in the same molecule?
    dists = np.linalg.norm(p - p_compare, axis=1) * mask
    r_bond = r + r_compare
    
    bond = np.where(np.logical_and(dists > 0.0001, dists < r_bond), 1, 0)
    
    source_row = source_row
    target_row = source_row + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_row = np.where(np.logical_or(target_row > len(structures), mask==0), len(structures), target_row) #If invalid target, write to dummy row
    
    source_atom = i_atom
    target_atom = i_atom + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_atom = np.where(np.logical_or(target_atom > max_atoms, mask==0), max_atoms, target_atom) #If invalid target, write to dummy col
    
    bonds[(source_row, target_atom)] = bond
    bonds[(target_row, source_atom)] = bond
    bond_dists[(source_row, target_atom)] = dists
    bond_dists[(target_row, source_atom)] = dists

bonds = np.delete(bonds, axis=0, obj=-1) #Delete dummy row
bonds = np.delete(bonds, axis=1, obj=-1) #Delete dummy col
bond_dists = np.delete(bond_dists, axis=0, obj=-1) #Delete dummy row
bond_dists = np.delete(bond_dists, axis=1, obj=-1) #Delete dummy col

print('Counting and condensing bonds')

bonds_numeric = [[i for i,x in enumerate(row) if x] for row in tqdm(bonds)]
bond_lengths = [[dist for i,dist in enumerate(row) if i in bonds_numeric[j]] for j,row in enumerate(tqdm(bond_dists))]
bond_lengths_mean = [ np.mean(x) for x in bond_lengths]
bond_lengths_std = [ np.std(x) for x in bond_lengths]
n_bonds = [len(x) for x in bonds_numeric]

#bond_data = {'bond_' + str(i):col for i, col in enumerate(np.transpose(bonds))}
#bond_data.update({'bonds_numeric':bonds_numeric, 'n_bonds':n_bonds})

bond_data = {'n_bonds':n_bonds, 'bond_lengths_mean': bond_lengths_mean,'bond_lengths_std':bond_lengths_std }
bond_df = pd.DataFrame(bond_data)
structures = structures.join(bond_df)
display(structures.head(20))

Calculating bonds



Counting and condensing bonds


,molecule_name,atom_index,atom,x,y,z,EN,rad,n_bonds,bond_lengths_mean,bond_lengths_std
0,dsgdb9nsd_000001,0,C,-0.0126981359,1.0858041580,0.0080009958,2.55,0.82,4,1.091949701309204,0.000002762467830
1,dsgdb9nsd_000001,1,H,0.0021504160,-0.0060313176,0.0019761204,2.20,0.43,1,1.091953039169312,0.000000000000000
2,dsgdb9nsd_000001,2,H,1.0117308430,1.4637511620,0.0002765748,2.20,0.43,1,1.091951608657837,0.000000000000000
3,dsgdb9nsd_000001,3,H,-0.5408150690,1.4475266140,-0.8766437152,2.20,0.43,1,1.091946363449097,0.000000000000000
4,dsgdb9nsd_000001,4,H,-0.5238136345,1.4379326440,0.9063972942,2.20,0.43,1,1.091947555541992,0.000000000000000
5,dsgdb9nsd_000002,0,N,-0.0404260543,1.0241077530,0.0625637998,3.04,0.80,3,1.017194986343384,0.000009144474461
6,dsgdb9nsd_000002,1,H,0.0172574639,0.0125452063,-0.0273771593,2.20,0.43,1,1.017189979553223,0.000000000000000
7,dsgdb9nsd_000002,2,H,0.9157893661,1.3587451950,-0.0287577581,2.20,0.43,1,1.017187237739563,0.000000000000000
8,dsgdb9nsd_000002,3,H,-0.5202777357,1.3435321260,-0.7755426124,2.20,0.43,1,1.017207860946655,0.000000000000000
9,dsgdb9nsd_000003,0,O,-0.0343604951,0.9775395708,0.0076015923,3.44,0.78,2,0.962106823921204,0.000000000000000


In [23]:
train.shape

(4658147, 10)

In [24]:
def map_atom_info(df, df2, atom_idx):
    df = pd.merge(df, df2, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    #df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, structures, 0)
test = map_atom_info(test, structures, 0)

# adding extra features
structures['c_x'] = structures.groupby('molecule_name')['x'].transform('mean')
structures['c_y']=structures.groupby('molecule_name')['y'].transform('mean')
structures['c_z']=structures.groupby('molecule_name')['z'].transform('mean')
structures['atom_n']=structures.groupby('molecule_name')['atom_index'].transform('max')

train = map_atom_info(train, structures, 1)
test = map_atom_info(test, structures, 1)

In [25]:
train.shape

(4658147, 34)

Let's get the distance between atoms first.

In [26]:
train_p_0 = train[['x_0', 'y_0', 'z_0']].values
train_p_1 = train[['x_1', 'y_1', 'z_1']].values
test_p_0 = test[['x_0', 'y_0', 'z_0']].values
test_p_1 = test[['x_1', 'y_1', 'z_1']].values

train['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)
test['dist'] = np.linalg.norm(test_p_0 - test_p_1, axis=1)
train['dist_x'] = (train['x_0'] - train['x_1']) ** 2
test['dist_x'] = (test['x_0'] - test['x_1']) ** 2
train['dist_y'] = (train['y_0'] - train['y_1']) ** 2
test['dist_y'] = (test['y_0'] - test['y_1']) ** 2
train['dist_z'] = (train['z_0'] - train['z_1']) ** 2
test['dist_z'] = (test['z_0'] - test['z_1']) ** 2

train['type_0'] = train['type'].apply(lambda x: x[0])
test['type_0'] = test['type'].apply(lambda x: x[0])


In [27]:
def create_features(df):
    df['molecule_couples'] = df.groupby('molecule_name')['id'].transform('count')
    df['molecule_dist_mean'] = df.groupby('molecule_name')['dist'].transform('mean')
    df['molecule_dist_min'] = df.groupby('molecule_name')['dist'].transform('min')
    df['molecule_dist_max'] = df.groupby('molecule_name')['dist'].transform('max')
    df['atom_0_couples_count'] = df.groupby(['molecule_name', 'atom_index_0'])['id'].transform('count')
    df['atom_1_couples_count'] = df.groupby(['molecule_name', 'atom_index_1'])['id'].transform('count')
    df[f'molecule_atom_index_0_x_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['x_1'].transform('std')
    df[f'molecule_atom_index_0_y_1_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('mean')
    df[f'molecule_atom_index_0_y_1_mean_diff'] = df[f'molecule_atom_index_0_y_1_mean'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_mean_div'] = df[f'molecule_atom_index_0_y_1_mean'] / df['y_1']
    df[f'molecule_atom_index_0_y_1_max'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('max')
    df[f'molecule_atom_index_0_y_1_max_diff'] = df[f'molecule_atom_index_0_y_1_max'] - df['y_1']
    df[f'molecule_atom_index_0_y_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['y_1'].transform('std')
    df[f'molecule_atom_index_0_z_1_std'] = df.groupby(['molecule_name', 'atom_index_0'])['z_1'].transform('std')
    df[f'molecule_atom_index_0_dist_mean'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('mean')
    df[f'molecule_atom_index_0_dist_mean_diff'] = df[f'molecule_atom_index_0_dist_mean'] - df['dist']
    df[f'molecule_atom_index_0_dist_mean_div'] = df[f'molecule_atom_index_0_dist_mean'] / df['dist']
    df[f'molecule_atom_index_0_dist_max'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('max')
    df[f'molecule_atom_index_0_dist_max_diff'] = df[f'molecule_atom_index_0_dist_max'] - df['dist']
    df[f'molecule_atom_index_0_dist_max_div'] = df[f'molecule_atom_index_0_dist_max'] / df['dist']
    df[f'molecule_atom_index_0_dist_min'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('min')
    df[f'molecule_atom_index_0_dist_min_diff'] = df[f'molecule_atom_index_0_dist_min'] - df['dist']
    df[f'molecule_atom_index_0_dist_min_div'] = df[f'molecule_atom_index_0_dist_min'] / df['dist']
    df[f'molecule_atom_index_0_dist_std'] = df.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('std')
    df[f'molecule_atom_index_0_dist_std_diff'] = df[f'molecule_atom_index_0_dist_std'] - df['dist']
    df[f'molecule_atom_index_0_dist_std_div'] = df[f'molecule_atom_index_0_dist_std'] / df['dist']
    df[f'molecule_atom_index_1_dist_mean'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('mean')
    df[f'molecule_atom_index_1_dist_mean_diff'] = df[f'molecule_atom_index_1_dist_mean'] - df['dist']
    df[f'molecule_atom_index_1_dist_mean_div'] = df[f'molecule_atom_index_1_dist_mean'] / df['dist']
    df[f'molecule_atom_index_1_dist_max'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('max')
    df[f'molecule_atom_index_1_dist_max_diff'] = df[f'molecule_atom_index_1_dist_max'] - df['dist']
    df[f'molecule_atom_index_1_dist_max_div'] = df[f'molecule_atom_index_1_dist_max'] / df['dist']
    df[f'molecule_atom_index_1_dist_min'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('min')
    df[f'molecule_atom_index_1_dist_min_diff'] = df[f'molecule_atom_index_1_dist_min'] - df['dist']
    df[f'molecule_atom_index_1_dist_min_div'] = df[f'molecule_atom_index_1_dist_min'] / df['dist']
    df[f'molecule_atom_index_1_dist_std'] = df.groupby(['molecule_name', 'atom_index_1'])['dist'].transform('std')
    df[f'molecule_atom_index_1_dist_std_diff'] = df[f'molecule_atom_index_1_dist_std'] - df['dist']
    df[f'molecule_atom_index_1_dist_std_div'] = df[f'molecule_atom_index_1_dist_std'] / df['dist']
    df[f'molecule_atom_1_dist_mean'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('mean')
    df[f'molecule_atom_1_dist_min'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('min')
    df[f'molecule_atom_1_dist_min_diff'] = df[f'molecule_atom_1_dist_min'] - df['dist']
    df[f'molecule_atom_1_dist_min_div'] = df[f'molecule_atom_1_dist_min'] / df['dist']
    df[f'molecule_atom_1_dist_std'] = df.groupby(['molecule_name', 'atom_1'])['dist'].transform('std')
    df[f'molecule_atom_1_dist_std_diff'] = df[f'molecule_atom_1_dist_std'] - df['dist']
    df[f'molecule_type_0_dist_std'] = df.groupby(['molecule_name', 'type_0'])['dist'].transform('std')
    df[f'molecule_type_0_dist_std_diff'] = df[f'molecule_type_0_dist_std'] - df['dist']
    df[f'molecule_type_dist_mean'] = df.groupby(['molecule_name', 'type'])['dist'].transform('mean')
    df[f'molecule_type_dist_mean_diff'] = df[f'molecule_type_dist_mean'] - df['dist']
    df[f'molecule_type_dist_mean_div'] = df[f'molecule_type_dist_mean'] / df['dist']
    df[f'molecule_type_dist_max'] = df.groupby(['molecule_name', 'type'])['dist'].transform('max')
    df[f'molecule_type_dist_min'] = df.groupby(['molecule_name', 'type'])['dist'].transform('min')
    df[f'molecule_type_dist_std'] = df.groupby(['molecule_name', 'type'])['dist'].transform('std')
    df[f'molecule_type_dist_std_diff'] = df[f'molecule_type_dist_std'] - df['dist']
    
    df['dot_product'] = (df[['x_0', 'y_0', 'z_0']].values*df[['x_1', 'y_1', 'z_1']].values).sum(axis=1)
    df = reduce_mem_usage(df)
    return df

train = create_features(train)
test = create_features(test)

Mem. usage decreased to 990.65 Mb (70.3% reduction)
Mem. usage decreased to 499.40 Mb (70.6% reduction)


In [28]:
train.shape

(4658147, 93)

In [29]:
def map_atom_info(df_1,df_2, atom_idx):
    df = pd.merge(df_1, df_2, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    df = df.drop('atom_index', axis=1)

    return df



def get_dist(df):
    df_temp=df.loc[:,["molecule_name","atom_index_0","atom_index_1","dist","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()
    df_temp_=df_temp.copy()
    df_temp_= df_temp_.rename(columns={'atom_index_0': 'atom_index_1',
                                       'atom_index_1': 'atom_index_0',
                                       'x_0': 'x_1',
                                       'y_0': 'y_1',
                                       'z_0': 'z_1',
                                       'x_1': 'x_0',
                                       'y_1': 'y_0',
                                       'z_1': 'z_0'})
    df_temp_all=pd.concat((df_temp,df_temp_),axis=0)

    df_temp_all["min_distance"]=df_temp_all.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('min')
    df_temp_all["max_distance"]=df_temp_all.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('max')
    
    df_temp= df_temp_all[df_temp_all["min_distance"]==df_temp_all["dist"]]
    df_temp=df_temp.drop(['x_0','y_0','z_0','min_distance'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_closest',
                                         'dist': 'distance_closest',
                                         'x_1': 'x_closest',
                                         'y_1': 'y_closest',
                                         'z_1': 'z_closest'})
    
    df_temp=df_temp.drop_duplicates(subset=['molecule_name', 'atom_index'])

    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_closest': f'atom_index_closest_{atom_idx}',
                                        'distance_closest': f'distance_closest_{atom_idx}',
                                        'x_closest': f'x_closest_{atom_idx}',
                                        'y_closest': f'y_closest_{atom_idx}',
                                        'z_closest': f'z_closest_{atom_idx}'})
        
        
        
    df_temp= df_temp_all[df_temp_all["max_distance"]==df_temp_all["dist"]].copy()
    df_temp=df_temp.drop(['x_0','y_0','z_0','max_distance'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_farthest',
                                         'dist': 'distance_farthest',
                                         'x_1': 'x_farthest',
                                         'y_1': 'y_farthest',
                                         'z_1': 'z_farthest'})
    
    df_temp=df_temp.drop_duplicates(subset=['molecule_name', 'atom_index'])

        
    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_farthest': f'atom_index_farthest_{atom_idx}',
                                        'distance_farthest': f'distance_farthest_{atom_idx}',
                                        'x_farthest': f'x_farthest_{atom_idx}',
                                        'y_farthest': f'y_farthest_{atom_idx}',
                                        'z_farthest': f'z_farthest_{atom_idx}'})
    return df

train = get_dist(train)
test = get_dist(test)
#print('dtrain size',dtrain.shape)
#print('dtest size',dtest.shape)

In [30]:
train.shape

(4658147, 117)

### cosine angles calculation

In [31]:
def add_features(df):
    df["distance_center0"]=((df['x_0']-df['c_x'])**2+(df['y_0']-df['c_y'])**2+(df['z_0']-df['c_z'])**2)**(1/2)
    df["distance_center1"]=((df['x_0']-df['c_x'])**2+(df['y_0']-df['c_y'])**2+(df['z_0']-df['c_z'])**2)**(1/2)
    df["distance_c0"]=((df['x_0']-df['x_closest_0'])**2+(df['y_0']-df['y_closest_0'])**2+(df['z_0']-df['z_closest_0'])**2)**(1/2)
    df["distance_c1"]=((df['x_1']-df['x_closest_1'])**2+(df['y_1']-df['y_closest_1'])**2+(df['z_1']-df['z_closest_1'])**2)**(1/2)
    df["distance_f0"]=((df['x_0']-df['x_farthest_0'])**2+(df['y_0']-df['y_farthest_0'])**2+(df['z_0']-df['z_farthest_0'])**2)**(1/2)
    df["distance_f1"]=((df['x_1']-df['x_farthest_1'])**2+(df['y_1']-df['y_farthest_1'])**2+(df['z_1']-df['z_farthest_1'])**2)**(1/2)
    df["vec_center0_x"]=(df['x_0']-df['c_x'])/(df["distance_center0"]+1e-10)
    df["vec_center0_y"]=(df['y_0']-df['c_y'])/(df["distance_center0"]+1e-10)
    df["vec_center0_z"]=(df['z_0']-df['c_z'])/(df["distance_center0"]+1e-10)
    df["vec_center1_x"]=(df['x_1']-df['c_x'])/(df["distance_center1"]+1e-10)
    df["vec_center1_y"]=(df['y_1']-df['c_y'])/(df["distance_center1"]+1e-10)
    df["vec_center1_z"]=(df['z_1']-df['c_z'])/(df["distance_center1"]+1e-10)
    df["vec_c0_x"]=(df['x_0']-df['x_closest_0'])/(df["distance_c0"]+1e-10)
    df["vec_c0_y"]=(df['y_0']-df['y_closest_0'])/(df["distance_c0"]+1e-10)
    df["vec_c0_z"]=(df['z_0']-df['z_closest_0'])/(df["distance_c0"]+1e-10)
    df["vec_c1_x"]=(df['x_1']-df['x_closest_1'])/(df["distance_c1"]+1e-10)
    df["vec_c1_y"]=(df['y_1']-df['y_closest_1'])/(df["distance_c1"]+1e-10)
    df["vec_c1_z"]=(df['z_1']-df['z_closest_1'])/(df["distance_c1"]+1e-10)
    df["vec_f0_x"]=(df['x_0']-df['x_farthest_0'])/(df["distance_f0"]+1e-10)
    df["vec_f0_y"]=(df['y_0']-df['y_farthest_0'])/(df["distance_f0"]+1e-10)
    df["vec_f0_z"]=(df['z_0']-df['z_farthest_0'])/(df["distance_f0"]+1e-10)
    df["vec_f1_x"]=(df['x_1']-df['x_farthest_1'])/(df["distance_f1"]+1e-10)
    df["vec_f1_y"]=(df['y_1']-df['y_farthest_1'])/(df["distance_f1"]+1e-10)
    df["vec_f1_z"]=(df['z_1']-df['z_farthest_1'])/(df["distance_f1"]+1e-10)
    df["vec_x"]=(df['x_1']-df['x_0'])/df["dist"]
    df["vec_y"]=(df['y_1']-df['y_0'])/df["dist"]
    df["vec_z"]=(df['z_1']-df['z_0'])/df["dist"]
    df["cos_c0_c1"]=df["vec_c0_x"]*df["vec_c1_x"]+df["vec_c0_y"]*df["vec_c1_y"]+df["vec_c0_z"]*df["vec_c1_z"]
    df["cos_f0_f1"]=df["vec_f0_x"]*df["vec_f1_x"]+df["vec_f0_y"]*df["vec_f1_y"]+df["vec_f0_z"]*df["vec_f1_z"]
    df["cos_center0_center1"]=df["vec_center0_x"]*df["vec_center1_x"]+df["vec_center0_y"]*df["vec_center1_y"]+df["vec_center0_z"]*df["vec_center1_z"]
    df["cos_c0"]=df["vec_c0_x"]*df["vec_x"]+df["vec_c0_y"]*df["vec_y"]+df["vec_c0_z"]*df["vec_z"]
    df["cos_c1"]=df["vec_c1_x"]*df["vec_x"]+df["vec_c1_y"]*df["vec_y"]+df["vec_c1_z"]*df["vec_z"]
    df["cos_f0"]=df["vec_f0_x"]*df["vec_x"]+df["vec_f0_y"]*df["vec_y"]+df["vec_f0_z"]*df["vec_z"]
    df["cos_f1"]=df["vec_f1_x"]*df["vec_x"]+df["vec_f1_y"]*df["vec_y"]+df["vec_f1_z"]*df["vec_z"]
    df["cos_center0"]=df["vec_center0_x"]*df["vec_x"]+df["vec_center0_y"]*df["vec_y"]+df["vec_center0_z"]*df["vec_z"]
    df["cos_center1"]=df["vec_center1_x"]*df["vec_x"]+df["vec_center1_y"]*df["vec_y"]+df["vec_center1_z"]*df["vec_z"]
    df=df.drop(['vec_c0_x','vec_c0_y','vec_c0_z','vec_c1_x','vec_c1_y','vec_c1_z',
                'vec_f0_x','vec_f0_y','vec_f0_z','vec_f1_x','vec_f1_y','vec_f1_z',
                'vec_center0_x','vec_center0_y','vec_center0_z','vec_center1_x','vec_center1_y','vec_center1_z',
                'vec_x','vec_y','vec_z'], axis=1)
    return df
    
train = reduce_mem_usage(add_features(train))
test = reduce_mem_usage(add_features(test))

Mem. usage decreased to 1319.38 Mb (0.0% reduction)
Mem. usage decreased to 676.22 Mb (0.0% reduction)


Dropping molecule_name and encode atom_0, atom_1 and type_0.<br>
**@TODO:** Try others encoders 

In [32]:
# del_cols_list = ['id','molecule_name','sd','pso','dso']
del_cols_list = ['id','molecule_name','pso']
def del_cols(df, cols):
    del_cols_list_ = [l for l in del_cols_list if l in df]
    df = df.drop(del_cols_list_,axis=1)
    return df

train = del_cols(train,del_cols_list)
test = del_cols(test,del_cols_list)

In [33]:
def encode_categoric_single(df):
    lbl = LabelEncoder()
    cat_cols=[]
    try:
        cat_cols = df.describe(include=['O']).columns.tolist()
        for cat in cat_cols:
            df[cat] = lbl.fit_transform(list(df[cat].values))
    except Exception as e:
        print('error: ', str(e) )

    return df

In [34]:
def encode_categoric(dtrain,dtest):
    lbl = LabelEncoder()
    objs_n = len(dtrain)
    dfmerge = pd.concat(objs=[dtrain,dtest],axis=0)
    cat_cols=[]
    try:
        cat_cols = dfmerge.describe(include=['O']).columns.tolist()
        for cat in cat_cols:
            dfmerge[cat] = lbl.fit_transform(list(dfmerge[cat].values))
    except Exception as e:
        print('error: ', str(e) )

    dtrain = dfmerge[:objs_n]
    dtest = dfmerge[objs_n:]
    return dtrain,dtest



In [35]:
train = encode_categoric_single(train)
test = encode_categoric_single(test)

In [36]:
y_fc = train['fc']
X = train.drop(['scalar_coupling_constant','fc', 'sd', 'dso'],axis=1)
y = train['scalar_coupling_constant']

X_test = test.copy()

In [37]:
print('X size',X.shape)
print('X_test size',X_test.shape)
print('dtest size',test.shape)
print('y_fc size',y_fc.shape)

# del train, test
# gc.collect()


X size (4658147, 125)
X_test size (2505542, 125)
dtest size (2505542, 125)
y_fc size (4658147,)


In [38]:
# good_columns = ['type',
#  'dot_product',
#  'bond_lengths_mean_y',
#  'bond_lengths_std_y',
#  'bond_lengths_mean_x',
#  'molecule_atom_index_0_dist_min_div',
#  'molecule_atom_index_0_dist_std_div',
#  'molecule_atom_index_0_dist_mean',
#  'molecule_atom_index_0_dist_max',
#  'dist_y',
#  'molecule_atom_index_1_dist_std_diff',
#  'z_0',
#  'molecule_type_dist_min',
#  'molecule_atom_index_0_y_1_mean_div',
#  'dist_x',
#  'x_0',
#  'y_0',
#  'molecule_type_dist_std',
#  'molecule_atom_index_0_y_1_std',
#  'molecule_dist_mean',
#  'molecule_atom_index_0_dist_std_diff',
#  'dist_z',
#  'molecule_atom_index_0_dist_std',
#  'molecule_atom_index_0_x_1_std',
#  'molecule_type_dist_std_diff',
#  'molecule_type_0_dist_std',
#  'dist',
#  'molecule_atom_index_0_dist_mean_diff',
#  'molecule_atom_index_1_dist_min_div',
#  'molecule_atom_index_1_dist_mean_diff',
#  'y_1',
#  'molecule_type_dist_mean_div',
#  'molecule_dist_max',
#  'molecule_atom_index_0_dist_mean_div',
#  'z_1',
#  'molecule_atom_index_0_z_1_std',
#  'molecule_atom_index_1_dist_mean_div',
#  'molecule_atom_index_1_dist_min_diff',
#  'molecule_atom_index_1_dist_mean',
#  'molecule_atom_index_1_dist_min',
#  'molecule_atom_index_1_dist_max',
#  'molecule_type_0_dist_std_diff',
#  'molecule_atom_index_0_dist_min_diff',
#  'molecule_type_dist_mean_diff',
#  'x_1',
#  'molecule_atom_index_0_y_1_max',
#  'molecule_atom_index_0_y_1_mean_diff',
#  'molecule_atom_1_dist_std_diff',
#  'molecule_atom_index_0_y_1_mean',
#  'molecule_atom_1_dist_std',
#  'molecule_type_dist_max']

# updated good_columns from angle calculations
# good_columns = ['type', 'bond_lengths_mean_y',
#  'bond_lengths_std_y',
#  'bond_lengths_mean_x',
#  'molecule_atom_index_0_dist_max',
#  'molecule_atom_index_0_dist_mean',
#  'molecule_atom_index_1_dist_min',
#  'dot_product',
#  'molecule_atom_index_0_dist_std',
#  'molecule_couples',
#  'cos_center1',
#  'molecule_atom_index_1_dist_max',
#  'cos_center0',
#  'distance_center0',
#  'molecule_atom_index_0_y_1_std',
#  'molecule_atom_index_0_dist_max_diff',
#  'dist_y',
#  'molecule_dist_mean',
#  'molecule_atom_index_1_dist_std',
#  'cos_center0_center1',
#  'molecule_atom_index_0_dist_std_div',
#  'molecule_dist_max',
#  'molecule_atom_index_0_dist_max_div',
#  'molecule_atom_index_0_dist_std_diff',
#  'y_0',
#  'molecule_atom_index_0_z_1_std',
#  'molecule_atom_index_1_dist_mean',
#  'molecule_atom_index_1_dist_std_diff',
#  'molecule_type_0_dist_std',
#  'molecule_atom_index_0_x_1_std',
#  'molecule_type_dist_max',
#  'molecule_atom_index_0_dist_min_div',
#  'molecule_atom_index_0_dist_min',
#  'molecule_atom_index_0_y_1_mean_div',
#  'molecule_atom_1_dist_mean',
#  'dist_z',
#  'molecule_type_dist_std',
#  'molecule_type_dist_std_diff',
#  'molecule_type_dist_mean_diff',
#  'molecule_atom_index_0_dist_mean_div',
#  'molecule_atom_index_0_y_1_mean_diff',
#  'dist',
#  'molecule_atom_index_1_dist_std_div',
#  'dist_x',
#  'molecule_atom_1_dist_std',
#  'molecule_atom_index_0_dist_mean_diff',
#  'x_0',
#  'molecule_atom_1_dist_std_diff',
#  'molecule_type_0_dist_std_diff',
#  'molecule_atom_index_1_dist_min_diff',
#  'molecule_atom_1_dist_min_diff',
#  'molecule_atom_index_0_y_1_max_diff',
#  'molecule_atom_1_dist_min',
#  'molecule_dist_min',
#  'molecule_type_dist_mean_div',
#  'z_0',
#  'molecule_atom_index_1_dist_max_diff',
#  'c_y',
#  'molecule_type_dist_min',
#  'c_x',
#  'x_1',
#  'molecule_type_dist_mean',
#  'y_1',
#  'molecule_atom_index_0_y_1_mean',
#  'c_z',
#  'atom_index_0',
#  'molecule_atom_index_1_dist_max_div',
#  'z_1',
#  'molecule_atom_index_1_dist_mean_diff',
#  'molecule_atom_index_1_dist_mean_div',
#  'molecule_atom_index_0_y_1_max']

### updated again after correct angle calculations
good_columns = ['type', 'bond_lengths_mean_y',
 'bond_lengths_std_y',
 'cos_f0',
 'cos_c0',
 'bond_lengths_mean_x',
 'cos_c0_c1',
 'molecule_atom_index_0_dist_mean',
 'cos_f1',
 'cos_c1',
 'molecule_atom_index_0_dist_std',
 'cos_center1',
 'distance_center0',
 'molecule_atom_index_0_dist_max',
 'cos_f0_f1',
 'molecule_dist_mean',
 'cos_center0',
 'molecule_couples',
 'molecule_atom_index_1_dist_std',
 'molecule_atom_index_0_dist_max_diff',
 'dot_product',
 'molecule_atom_index_1_dist_min',
 'molecule_atom_index_1_dist_mean',
 'molecule_atom_1_dist_mean',
 'molecule_atom_index_0_y_1_std',
 'molecule_dist_max',
 'molecule_atom_index_1_dist_std_diff',
 'molecule_atom_index_0_dist_max_div',
 'molecule_atom_index_0_dist_std_div',
 'cos_center0_center1',
 'molecule_atom_index_0_z_1_std',
 'molecule_type_0_dist_std',
 'molecule_atom_index_0_dist_mean_diff',
 'dist_y',
 'molecule_atom_index_0_dist_mean_div',
 'distance_c0',
 'molecule_type_dist_max',
 'molecule_type_dist_std',
 'molecule_atom_1_dist_std',
 'dist',
 'molecule_atom_index_0_x_1_std',
 'molecule_type_dist_std_diff',
 'molecule_atom_index_0_dist_std_diff',
 'distance_f0',
 'molecule_type_0_dist_std_diff',
 'molecule_atom_index_1_dist_std_div',
 'molecule_type_dist_mean_diff',
 'molecule_atom_1_dist_std_diff',
 'molecule_atom_index_0_dist_min_div',
 'molecule_atom_index_1_dist_max',
 'dist_z',
 'molecule_dist_min',
 'molecule_atom_index_0_y_1_mean_div',
 'molecule_atom_index_1_dist_max_diff',
 'molecule_atom_1_dist_min',
 'y_farthest_0',
 'molecule_atom_index_1_dist_mean_diff',
 'molecule_type_dist_min',
 'molecule_atom_index_1_dist_min_diff',
 'molecule_type_dist_mean',
 'molecule_type_dist_mean_div',
 'dist_x',
 'molecule_atom_index_0_y_1_max_diff',
 'molecule_atom_index_0_dist_min',
 'molecule_atom_1_dist_min_diff',
 'molecule_atom_index_1_dist_mean_div',
 'c_y',
 'molecule_atom_index_0_y_1_mean_diff',
 'x_farthest_0',
 'molecule_atom_index_1_dist_max_div',
 'y_0']

if work == "predicting":
    good_columns = X.columns.values  # using all features and not just those above
    X = X[good_columns].copy()
    X_test = X_test[good_columns].copy()

<a id="id6"></a> <br> 
# **6. Model** 


In [39]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)

## Create out of fold feature

In [40]:
params = {'num_leaves': 50,
          'min_child_samples': 79,
          'min_data_in_leaf': 100,
          'objective': 'regression',
          'max_depth': 9,
          'learning_rate': 0.2,
          "boosting_type": "gbdt",
          "subsample_freq": 1,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'reg_alpha': 0.1,
          'reg_lambda': 0.3,
          'colsample_bytree': 1.0
         }

In [41]:
if work == "predicting":
    X_short = pd.DataFrame({'ind': list(X.index), 'type': X['type'].values, 'oof': [0] * len(X), 'target': y_fc.values})
    X_short_test = pd.DataFrame({'ind': list(X_test.index), 'type': X_test['type'].values, 'prediction': [0] * len(X_test)})
    result_dict_lgb_oof =  {}
    for t in X['type'].unique():
        print(f'Training of type {t}')
        X_t = X.loc[X['type'] == t]
        X_test_t = X_test.loc[X_test['type'] == t]
        y_t = X_short.loc[X_short['type'] == t, 'target']
        result_dict_lgb_oof = train_model_regression(X=X_t, X_test=X_test_t, y=y_t, params=params, folds=folds, model_type='lgb', eval_metric='group_mae', plot_feature_importance=False,
                                                          verbose=500, early_stopping_rounds=200, n_estimators=n_estimators_default)
        X_short.loc[X_short['type'] == t, 'oof'] = result_dict_lgb_oof['oof']
        X_short_test.loc[X_short_test['type'] == t, 'prediction'] = result_dict_lgb_oof['prediction']

Training of type 0
Fold 1 started at Tue Jul 30 23:19:42 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 1.36853	valid_1's l1: 1.50258
[1000]	training's l1: 1.15178	valid_1's l1: 1.37537
[1500]	training's l1: 1.01512	valid_1's l1: 1.31196
[2000]	training's l1: 0.913519	valid_1's l1: 1.27256
[2500]	training's l1: 0.831517	valid_1's l1: 1.24456
[3000]	training's l1: 0.763594	valid_1's l1: 1.2253
[3500]	training's l1: 0.705038	valid_1's l1: 1.20998
[4000]	training's l1: 0.653759	valid_1's l1: 1.19747
[4500]	training's l1: 0.60811	valid_1's l1: 1.1876
[5000]	training's l1: 0.567591	valid_1's l1: 1.18013
[5500]	training's l1: 0.530489	valid_1's l1: 1.17331
[6000]	training's l1: 0.496973	valid_1's l1: 1.16771
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.496973	valid_1's l1: 1.16771
Fold 2 started at Tue Jul 30 23:26:53 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 1.37168	valid_1's 

[4500]	training's l1: 0.091164	valid_1's l1: 0.288292
[5000]	training's l1: 0.0822051	valid_1's l1: 0.287052
[5500]	training's l1: 0.074359	valid_1's l1: 0.286051
[6000]	training's l1: 0.0674873	valid_1's l1: 0.285271
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.0674873	valid_1's l1: 0.285271
CV mean score: -1.2553, std: 0.0031.
Training of type 1
Fold 1 started at Wed Jul 31 00:18:10 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.257308	valid_1's l1: 0.589902
[1000]	training's l1: 0.129624	valid_1's l1: 0.576521
[1500]	training's l1: 0.0704558	valid_1's l1: 0.571581
[2000]	training's l1: 0.0402666	valid_1's l1: 0.569867
[2500]	training's l1: 0.0238236	valid_1's l1: 0.569185
[3000]	training's l1: 0.0145593	valid_1's l1: 0.568878
[3500]	training's l1: 0.0092158	valid_1's l1: 0.568758
Early stopping, best iteration is:
[3798]	training's l1: 0.00715662	valid_1's l1: 0.568649
Fold 2 started at Wed Jul 31 00:18:54 2019
T

[500]	training's l1: 0.675882	valid_1's l1: 0.723057
[1000]	training's l1: 0.575343	valid_1's l1: 0.650935
[1500]	training's l1: 0.514971	valid_1's l1: 0.614918
[2000]	training's l1: 0.47096	valid_1's l1: 0.592649
[2500]	training's l1: 0.436501	valid_1's l1: 0.577134
[3000]	training's l1: 0.40751	valid_1's l1: 0.565678
[3500]	training's l1: 0.382302	valid_1's l1: 0.556431
[4000]	training's l1: 0.359963	valid_1's l1: 0.54878
[4500]	training's l1: 0.340263	valid_1's l1: 0.542618
[5000]	training's l1: 0.322298	valid_1's l1: 0.537076
[5500]	training's l1: 0.306036	valid_1's l1: 0.53273
[6000]	training's l1: 0.291154	valid_1's l1: 0.529198
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.291154	valid_1's l1: 0.529198
Fold 2 started at Wed Jul 31 00:43:14 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.675651	valid_1's l1: 0.720326
[1000]	training's l1: 0.575524	valid_1's l1: 0.650508
[1500]	training's l1: 0.516391	valid_1's l

[5000]	training's l1: 0.108369	valid_1's l1: 0.253908
[5500]	training's l1: 0.100406	valid_1's l1: 0.25241
[6000]	training's l1: 0.0932201	valid_1's l1: 0.251066
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.0932201	valid_1's l1: 0.251066
CV mean score: -1.3795, std: 0.0035.
Training of type 5
Fold 1 started at Wed Jul 31 01:58:01 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.635836	valid_1's l1: 0.663691
[1000]	training's l1: 0.54689	valid_1's l1: 0.596136
[1500]	training's l1: 0.494051	valid_1's l1: 0.561404
[2000]	training's l1: 0.455923	valid_1's l1: 0.539287
[2500]	training's l1: 0.426964	valid_1's l1: 0.525094
[3000]	training's l1: 0.401815	valid_1's l1: 0.513606
[3500]	training's l1: 0.380313	valid_1's l1: 0.504503
[4000]	training's l1: 0.361511	valid_1's l1: 0.497309
[4500]	training's l1: 0.344753	valid_1's l1: 0.491156
[5000]	training's l1: 0.329216	valid_1's l1: 0.485676
[5500]	training's l1: 0.315665	vali

[500]	training's l1: 0.143117	valid_1's l1: 0.204497
[1000]	training's l1: 0.101031	valid_1's l1: 0.191153
[1500]	training's l1: 0.0763436	valid_1's l1: 0.185385
[2000]	training's l1: 0.0594615	valid_1's l1: 0.182527
[2500]	training's l1: 0.0472653	valid_1's l1: 0.180522
[3000]	training's l1: 0.0380639	valid_1's l1: 0.179326
[3500]	training's l1: 0.0309916	valid_1's l1: 0.17851
[4000]	training's l1: 0.0254697	valid_1's l1: 0.177909
[4500]	training's l1: 0.0210788	valid_1's l1: 0.177467
[5000]	training's l1: 0.0176003	valid_1's l1: 0.17723
[5500]	training's l1: 0.0147705	valid_1's l1: 0.176973
[6000]	training's l1: 0.012477	valid_1's l1: 0.176837
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.012477	valid_1's l1: 0.176837
CV mean score: -1.7330, std: 0.0035.


In [42]:
if work == "testing":
    result_dict_lgb2 = train_model_regression(X=X, X_test=X_test, y=y, params=params, folds=folds, model_type='lgb', eval_metric='group_mae', plot_feature_importance=True,
                                                         verbose=500, early_stopping_rounds=200, n_estimators=n_estimators_default)
    #Best Features? 
    feature_importance = result_dict_lgb2['feature_importance']
    best_features = feature_importance[['feature','importance']].groupby(['feature']).mean().sort_values(
            by='importance',ascending=False).iloc[:70,0:0].index.tolist()
    best_features

In [43]:
# sorted_feature_importance = feature_importance[['feature','importance']].groupby(['feature']).mean().sort_values(
#         by='importance',ascending=False)

In [44]:
# from matplotlib.pyplot import figure
# figure(figsize=(15,20))
# sns.barplot(x = 'importance' , y = 'feature', data = sorted_feature_importance)
# # plt.xticks(rotation = 90)
# plt.show()

In [45]:
X['oof_fc'] = X_short['oof']
X_test['oof_fc'] = X_short_test['prediction']

<a id="id7"></a> <br> 
# **7. Modelling sd and dso** 

In [46]:
y_sd = train['sd']
# X = X.drop(['scalar_coupling_constant','sd', 'dso'],axis=1)
# y = train['scalar_coupling_constant']

In [49]:

# good_columns = good_columns.append('oof_fc')
good_columns = list(good_columns)
good_columns.append('oof_fc')
X = X[good_columns].copy()
X_test = X_test[good_columns].copy()

In [50]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)

In [51]:
params = {'num_leaves': 50,
          'min_child_samples': 79,
          'min_data_in_leaf': 100,
          'objective': 'regression',
          'max_depth': 9,
          'learning_rate': 0.2,
          "boosting_type": "gbdt",
          "subsample_freq": 1,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'reg_alpha': 0.1,
          'reg_lambda': 0.3,
          'colsample_bytree': 1.0
         }

In [52]:
X_short = pd.DataFrame({'ind': list(X.index), 'type': X['type'].values, 'oof': [0] * len(X), 'target': y_sd.values})
X_short_test = pd.DataFrame({'ind': list(X_test.index), 'type': X_test['type'].values, 'prediction': [0] * len(X_test)})
result_dict_lgb_oof =  {}
for t in X['type'].unique():
    print(f'Training of type {t}')
    X_t = X.loc[X['type'] == t]
    X_test_t = X_test.loc[X_test['type'] == t]
    y_t = X_short.loc[X_short['type'] == t, 'target']
    result_dict_lgb_oof = train_model_regression(X=X_t, X_test=X_test_t, y=y_t, params=params, folds=folds, model_type='lgb', eval_metric='group_mae', plot_feature_importance=False,
                                                      verbose=500, early_stopping_rounds=200, n_estimators=n_estimators_default)
    X_short.loc[X_short['type'] == t, 'oof'] = result_dict_lgb_oof['oof']
    X_short_test.loc[X_short_test['type'] == t, 'prediction'] = result_dict_lgb_oof['prediction']

Training of type 0
Fold 1 started at Wed Jul 31 09:24:42 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.00851762	valid_1's l1: 0.00940613
[1000]	training's l1: 0.00719237	valid_1's l1: 0.0086298
[1500]	training's l1: 0.00633895	valid_1's l1: 0.00821562
[2000]	training's l1: 0.00571465	valid_1's l1: 0.0079655
[2500]	training's l1: 0.00523215	valid_1's l1: 0.00780551
[3000]	training's l1: 0.00481726	valid_1's l1: 0.00766976
[3500]	training's l1: 0.00446297	valid_1's l1: 0.00756416
[4000]	training's l1: 0.00415611	valid_1's l1: 0.00747315
[4500]	training's l1: 0.00388446	valid_1's l1: 0.00740342
[5000]	training's l1: 0.0036486	valid_1's l1: 0.00734989
[5500]	training's l1: 0.00343246	valid_1's l1: 0.00729829
[6000]	training's l1: 0.00324155	valid_1's l1: 0.00725707
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.00324155	valid_1's l1: 0.00725707
Fold 2 started at Wed Jul 31 09:32:34 2019
Training until validation scores d

Fold 5 started at Wed Jul 31 10:26:25 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.00407996	valid_1's l1: 0.00479117
[1000]	training's l1: 0.00328517	valid_1's l1: 0.00439833
[1500]	training's l1: 0.0028068	valid_1's l1: 0.00420782
[2000]	training's l1: 0.00247418	valid_1's l1: 0.00410856
[2500]	training's l1: 0.00222085	valid_1's l1: 0.00403835
[3000]	training's l1: 0.00201587	valid_1's l1: 0.00398858
[3500]	training's l1: 0.00184701	valid_1's l1: 0.00395257
[4000]	training's l1: 0.00170711	valid_1's l1: 0.0039262
[4500]	training's l1: 0.00158639	valid_1's l1: 0.00390582
[5000]	training's l1: 0.00148211	valid_1's l1: 0.00388743
[5500]	training's l1: 0.00139109	valid_1's l1: 0.00387107
[6000]	training's l1: 0.00130986	valid_1's l1: 0.003859
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.00130986	valid_1's l1: 0.003859
CV mean score: -5.5582, std: 0.0040.
Training of type 1
Fold 1 started at Wed Jul 31 10:31:57 2019


[6000]	training's l1: 0.000690376	valid_1's l1: 0.00386059
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.000690376	valid_1's l1: 0.00386059
CV mean score: -5.5429, std: 0.0082.
Training of type 2
Fold 1 started at Wed Jul 31 10:46:44 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.0100875	valid_1's l1: 0.0108642
[1000]	training's l1: 0.00862836	valid_1's l1: 0.0098492
[1500]	training's l1: 0.00772804	valid_1's l1: 0.0093061
[2000]	training's l1: 0.00708474	valid_1's l1: 0.00896716
[2500]	training's l1: 0.00656277	valid_1's l1: 0.00872786
[3000]	training's l1: 0.00614343	valid_1's l1: 0.0085597
[3500]	training's l1: 0.00577438	valid_1's l1: 0.00842636
[4000]	training's l1: 0.0054529	valid_1's l1: 0.00831417
[4500]	training's l1: 0.00516958	valid_1's l1: 0.00822091
[5000]	training's l1: 0.00491143	valid_1's l1: 0.00813784
[5500]	training's l1: 0.0046818	valid_1's l1: 0.00807444
[6000]	training's l1: 0.0044705	valid_1's 

[6000]	training's l1: 0.0013532	valid_1's l1: 0.00295044
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.0013532	valid_1's l1: 0.00295044
Fold 5 started at Wed Jul 31 12:14:45 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.00344763	valid_1's l1: 0.00386889
[1000]	training's l1: 0.00289262	valid_1's l1: 0.00353602
[1500]	training's l1: 0.00254288	valid_1's l1: 0.00336529
[2000]	training's l1: 0.00228761	valid_1's l1: 0.00325301
[2500]	training's l1: 0.00208759	valid_1's l1: 0.00317536
[3000]	training's l1: 0.00192374	valid_1's l1: 0.00311899
[3500]	training's l1: 0.00178588	valid_1's l1: 0.00307484
[4000]	training's l1: 0.00167059	valid_1's l1: 0.00304424
[4500]	training's l1: 0.00156914	valid_1's l1: 0.00301875
[5000]	training's l1: 0.00148071	valid_1's l1: 0.0029968
[5500]	training's l1: 0.0014026	valid_1's l1: 0.00297831
[6000]	training's l1: 0.00133356	valid_1's l1: 0.0029639
Did not meet early stopping. Best iterat

[5500]	training's l1: 0.000824818	valid_1's l1: 0.00392526
[6000]	training's l1: 0.000772877	valid_1's l1: 0.00391954
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.000772877	valid_1's l1: 0.00391954
Fold 4 started at Wed Jul 31 13:47:25 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.00338047	valid_1's l1: 0.00456394
[1000]	training's l1: 0.00253534	valid_1's l1: 0.00427539
[1500]	training's l1: 0.00204888	valid_1's l1: 0.00416056
[2000]	training's l1: 0.00172143	valid_1's l1: 0.00409336
[2500]	training's l1: 0.00148325	valid_1's l1: 0.00405148
[3000]	training's l1: 0.00130363	valid_1's l1: 0.00402611
[3500]	training's l1: 0.00116352	valid_1's l1: 0.00400601
[4000]	training's l1: 0.00105247	valid_1's l1: 0.00399176
[4500]	training's l1: 0.000961614	valid_1's l1: 0.00398007
[5000]	training's l1: 0.000886415	valid_1's l1: 0.00397187
[5500]	training's l1: 0.000824181	valid_1's l1: 0.00396564
[6000]	training's l1: 0.00077

In [53]:
X['oof_sd'] = X_short['oof']
X_test['oof_sd'] = X_short_test['prediction']

In [54]:
good_columns.append('oof_sd')

X = X[good_columns].copy()
X_test = X_test[good_columns].copy()

In [55]:
y_dso = train['dso']
del train, test
gc.collect()

1835

In [56]:
X_short = pd.DataFrame({'ind': list(X.index), 'type': X['type'].values, 'oof': [0] * len(X), 'target': y_dso.values})
X_short_test = pd.DataFrame({'ind': list(X_test.index), 'type': X_test['type'].values, 'prediction': [0] * len(X_test)})
result_dict_lgb_oof =  {}
for t in X['type'].unique():
    print(f'Training of type {t}')
    X_t = X.loc[X['type'] == t]
    X_test_t = X_test.loc[X_test['type'] == t]
    y_t = X_short.loc[X_short['type'] == t, 'target']
    result_dict_lgb_oof = train_model_regression(X=X_t, X_test=X_test_t, y=y_t, params=params, folds=folds, model_type='lgb', eval_metric='group_mae', plot_feature_importance=False,
                                                      verbose=500, early_stopping_rounds=200, n_estimators=n_estimators_default)
    X_short.loc[X_short['type'] == t, 'oof'] = result_dict_lgb_oof['oof']
    X_short_test.loc[X_short_test['type'] == t, 'prediction'] = result_dict_lgb_oof['prediction']

Training of type 0
Fold 1 started at Wed Jul 31 13:54:09 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.0128981	valid_1's l1: 0.014005
[1000]	training's l1: 0.0107909	valid_1's l1: 0.0126375
[1500]	training's l1: 0.0094868	valid_1's l1: 0.0119294
[2000]	training's l1: 0.0085536	valid_1's l1: 0.0115042
[2500]	training's l1: 0.00781479	valid_1's l1: 0.0112052
[3000]	training's l1: 0.00719691	valid_1's l1: 0.0109786
[3500]	training's l1: 0.00666924	valid_1's l1: 0.0108033
[4000]	training's l1: 0.00621587	valid_1's l1: 0.0106589
[4500]	training's l1: 0.00582005	valid_1's l1: 0.0105455
[5000]	training's l1: 0.00546258	valid_1's l1: 0.010452
[5500]	training's l1: 0.00514065	valid_1's l1: 0.0103694
[6000]	training's l1: 0.00484993	valid_1's l1: 0.0102954
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.00484993	valid_1's l1: 0.0102954
Fold 2 started at Wed Jul 31 14:02:00 2019
Training until validation scores don't improve for

[1000]	training's l1: 0.0249382	valid_1's l1: 0.0332927
[1500]	training's l1: 0.0211916	valid_1's l1: 0.0319375
[2000]	training's l1: 0.018422	valid_1's l1: 0.0311401
[2500]	training's l1: 0.0162158	valid_1's l1: 0.0306051
[3000]	training's l1: 0.0144087	valid_1's l1: 0.0302196
[3500]	training's l1: 0.0128892	valid_1's l1: 0.0299206
[4000]	training's l1: 0.0116006	valid_1's l1: 0.0297055
[4500]	training's l1: 0.0104847	valid_1's l1: 0.0295106
[5000]	training's l1: 0.00952441	valid_1's l1: 0.0293645
[5500]	training's l1: 0.00867457	valid_1's l1: 0.0292361
[6000]	training's l1: 0.0079367	valid_1's l1: 0.0291299
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.0079367	valid_1's l1: 0.0291299
CV mean score: -3.5334, std: 0.0026.
Training of type 1
Fold 1 started at Wed Jul 31 14:56:55 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.00181746	valid_1's l1: 0.00365771
[1000]	training's l1: 0.00125047	valid_1's l1: 0.00355738
[1

[3000]	training's l1: 0.00695421	valid_1's l1: 0.00911639
[3500]	training's l1: 0.00653242	valid_1's l1: 0.00892308
[4000]	training's l1: 0.0061668	valid_1's l1: 0.00876371
[4500]	training's l1: 0.00584506	valid_1's l1: 0.00864046
[5000]	training's l1: 0.00555434	valid_1's l1: 0.00852652
[5500]	training's l1: 0.00529081	valid_1's l1: 0.00843136
[6000]	training's l1: 0.00505572	valid_1's l1: 0.0083527
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.00505572	valid_1's l1: 0.0083527
Fold 2 started at Wed Jul 31 15:23:05 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.0116226	valid_1's l1: 0.012235
[1000]	training's l1: 0.00985241	valid_1's l1: 0.0108793
[1500]	training's l1: 0.00879356	valid_1's l1: 0.0101666
[2000]	training's l1: 0.00803212	valid_1's l1: 0.00970837
[2500]	training's l1: 0.0074377	valid_1's l1: 0.00938475
[3000]	training's l1: 0.00694769	valid_1's l1: 0.00914218
[3500]	training's l1: 0.00652124	valid_1's l

[4500]	training's l1: 0.011463	valid_1's l1: 0.0235545
[5000]	training's l1: 0.0106447	valid_1's l1: 0.0233585
[5500]	training's l1: 0.00990853	valid_1's l1: 0.0232063
[6000]	training's l1: 0.00925005	valid_1's l1: 0.023067
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.00925005	valid_1's l1: 0.023067
CV mean score: -3.7720, std: 0.0049.
Training of type 5
Fold 1 started at Wed Jul 31 16:45:36 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.0138326	valid_1's l1: 0.0143843
[1000]	training's l1: 0.0118194	valid_1's l1: 0.0127696
[1500]	training's l1: 0.0106723	valid_1's l1: 0.0119681
[2000]	training's l1: 0.00982522	valid_1's l1: 0.011426
[2500]	training's l1: 0.00915795	valid_1's l1: 0.0110223
[3000]	training's l1: 0.00861493	valid_1's l1: 0.0107393
[3500]	training's l1: 0.00814366	valid_1's l1: 0.0105063
[4000]	training's l1: 0.00773935	valid_1's l1: 0.0103167
[4500]	training's l1: 0.00738092	valid_1's l1: 0.0101607
[5

[4500]	training's l1: 0.000805345	valid_1's l1: 0.00290316
[5000]	training's l1: 0.000751025	valid_1's l1: 0.00289432
[5500]	training's l1: 0.00070651	valid_1's l1: 0.00288729
[6000]	training's l1: 0.000670445	valid_1's l1: 0.00288182
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.000670445	valid_1's l1: 0.00288182
Fold 5 started at Wed Jul 31 19:58:04 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.00260055	valid_1's l1: 0.00344085
[1000]	training's l1: 0.00196178	valid_1's l1: 0.00318498
[1500]	training's l1: 0.00160269	valid_1's l1: 0.0030744
[2000]	training's l1: 0.00136362	valid_1's l1: 0.00301562
[2500]	training's l1: 0.00119094	valid_1's l1: 0.0029778
[3000]	training's l1: 0.00105983	valid_1's l1: 0.0029516
[3500]	training's l1: 0.000957398	valid_1's l1: 0.00293342
[4000]	training's l1: 0.00087526	valid_1's l1: 0.00291893
[4500]	training's l1: 0.000808951	valid_1's l1: 0.00290908
[5000]	training's l1: 0.00075436

In [57]:
X['oof_dso'] = X_short['oof']
X_test['oof_dso'] = X_short_test['prediction']

<a id="id7"></a> <br> 
# **7. Final Model** 

In [58]:
good_columns.append('oof_dso')

X = X[good_columns].copy()
X_test = X_test[good_columns].copy()

In [59]:
params = {'num_leaves': 50,
          'min_child_samples': 79,
          'min_data_in_leaf': 100,
          'objective': 'regression',
          'max_depth': 9,
          'learning_rate': 0.2,
          "boosting_type": "gbdt",
          "subsample_freq": 1,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'mae',
          "verbosity": -1,
          'reg_alpha': 0.1,
          'reg_lambda': 0.3,
          'colsample_bytree': 1.0
         }

## Training models for each type

In [60]:
X_short = pd.DataFrame({'ind': list(X.index), 'type': X['type'].values, 'oof': [0] * len(X), 'target': y.values})
X_short_test = pd.DataFrame({'ind': list(X_test.index), 'type': X_test['type'].values, 'prediction': [0] * len(X_test)})
for t in X['type'].unique():
    print(f'Training of type {t}')
    X_t = X.loc[X['type'] == t]
    X_test_t = X_test.loc[X_test['type'] == t]
    y_t = X_short.loc[X_short['type'] == t, 'target']
    result_dict_lgb3 = train_model_regression(X=X_t, X_test=X_test_t, y=y_t, params=params, folds=folds, model_type='lgb', eval_metric='group_mae', plot_feature_importance=False,
                                                      verbose=500, early_stopping_rounds=200, n_estimators=n_estimators_default)
    X_short.loc[X_short['type'] == t, 'oof'] = result_dict_lgb3['oof']
    X_short_test.loc[X_short_test['type'] == t, 'prediction'] = result_dict_lgb3['prediction']

Training of type 0
Fold 1 started at Wed Jul 31 20:01:33 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 1.04001	valid_1's l1: 1.14823
[1000]	training's l1: 0.94965	valid_1's l1: 1.14272
[1500]	training's l1: 0.871406	valid_1's l1: 1.13502
[2000]	training's l1: 0.804329	valid_1's l1: 1.12915
[2500]	training's l1: 0.744693	valid_1's l1: 1.12355
[3000]	training's l1: 0.691831	valid_1's l1: 1.11828
[3500]	training's l1: 0.643848	valid_1's l1: 1.11288
[4000]	training's l1: 0.600622	valid_1's l1: 1.10789
[4500]	training's l1: 0.561276	valid_1's l1: 1.10305
[5000]	training's l1: 0.525244	valid_1's l1: 1.09897
[5500]	training's l1: 0.49243	valid_1's l1: 1.09566
[6000]	training's l1: 0.46194	valid_1's l1: 1.092
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.46194	valid_1's l1: 1.092
Fold 2 started at Wed Jul 31 20:08:40 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 1.03864	valid_1's l1:

[4500]	training's l1: 0.0746353	valid_1's l1: 0.238723
[5000]	training's l1: 0.0673016	valid_1's l1: 0.237839
[5500]	training's l1: 0.0608206	valid_1's l1: 0.237032
[6000]	training's l1: 0.0551592	valid_1's l1: 0.236446
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.0551592	valid_1's l1: 0.236446
CV mean score: -1.4356, std: 0.0056.
Training of type 1
Fold 1 started at Wed Jul 31 21:00:24 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[31]	training's l1: 0.510325	valid_1's l1: 0.553859
Fold 2 started at Wed Jul 31 21:00:27 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[28]	training's l1: 0.512175	valid_1's l1: 0.554512
Fold 3 started at Wed Jul 31 21:00:29 2019
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[28]	training's l1: 0.516613	valid_1's l1: 0.544041
Fold 4 started at Wed Jul 31 21:00:32 2019

Fold 4 started at Wed Jul 31 21:50:16 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.46061	valid_1's l1: 0.491655
[1000]	training's l1: 0.424962	valid_1's l1: 0.481678
[1500]	training's l1: 0.395803	valid_1's l1: 0.474889
[2000]	training's l1: 0.370491	valid_1's l1: 0.469061
[2500]	training's l1: 0.348444	valid_1's l1: 0.464619
[3000]	training's l1: 0.328313	valid_1's l1: 0.460183
[3500]	training's l1: 0.31018	valid_1's l1: 0.45662
[4000]	training's l1: 0.293314	valid_1's l1: 0.452927
[4500]	training's l1: 0.278188	valid_1's l1: 0.449942
[5000]	training's l1: 0.264484	valid_1's l1: 0.447689
[5500]	training's l1: 0.251636	valid_1's l1: 0.445403
[6000]	training's l1: 0.23964	valid_1's l1: 0.443326
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.23964	valid_1's l1: 0.443326
Fold 5 started at Wed Jul 31 22:02:41 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.459301	valid_1's l1: 

[3500]	training's l1: 0.321276	valid_1's l1: 0.429691
[4000]	training's l1: 0.30778	valid_1's l1: 0.42686
[4500]	training's l1: 0.294976	valid_1's l1: 0.42418
[5000]	training's l1: 0.283169	valid_1's l1: 0.42187
[5500]	training's l1: 0.272049	valid_1's l1: 0.419767
[6000]	training's l1: 0.261826	valid_1's l1: 0.417853
Did not meet early stopping. Best iteration is:
[6000]	training's l1: 0.261826	valid_1's l1: 0.417853
Fold 4 started at Wed Jul 31 23:36:04 2019
Training until validation scores don't improve for 200 rounds.
[500]	training's l1: 0.433243	valid_1's l1: 0.454823
[1000]	training's l1: 0.408578	valid_1's l1: 0.448455
[1500]	training's l1: 0.387309	valid_1's l1: 0.443157
[2000]	training's l1: 0.368546	valid_1's l1: 0.439128
[2500]	training's l1: 0.351228	valid_1's l1: 0.435252
[3000]	training's l1: 0.335678	valid_1's l1: 0.43209
[3500]	training's l1: 0.320996	valid_1's l1: 0.429061
[4000]	training's l1: 0.307536	valid_1's l1: 0.426485
[4500]	training's l1: 0.295015	valid_1's l

<a id="id8"></a> <br> 
# **8. Submittion** 

In [61]:
#Training models for type
sub['scalar_coupling_constant'] = X_short_test['prediction']
sub.to_csv('./predictions/all_features_prediction_v4.csv', index=False)
sub.head()


,id,scalar_coupling_constant
0,4658147,15.261980988424892
1,4658148,185.648680894095833
2,4658149,11.629185373590724
3,4658150,184.210998638186311
4,4658151,10.465041035742404


<a id="ref"></a> <br> 
# **8. References** 

[1] OOF Model: https://www.kaggle.com/adarshchavakula/out-of-fold-oof-model-cross-validation<br>
[2] Using Meta Features: https://www.kaggle.com/artgor/using-meta-features-to-improve-model<br>
[3] Lot of Features: https://towardsdatascience.com/understanding-feature-engineering-part-1-continuous-numeric-data-da4e47099a7b <br>
[4] Angle Feature: https://www.kaggle.com/kmat2019/effective-feature <br>
[5] Recovering bonds from structure: https://www.kaggle.com/aekoch95/bonds-from-structure-data <br>